In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from Drive import *

In [2]:
months=['January','February','March','April','May','June','July','August','September','October','November','December']

In [3]:
# Downloads the file from Gdrive
def fileDownloader():
    """Downloads Student Gradebook.xlsx from GDrive"""
    try:
        auth=GDriveAuthenticator.GDriveAuthenticator('Drive\\client_secret.json')
        drive=Gdrive.Gdrive(auth.authenticate())
        drive.download()
    except:
        print('Problem in DRIVE API usage')
        raise

In [4]:
# imports the data and merges into one dataframe and returns it
def data_importer():
    """imports the data and merges into one dataframe and returns it"""
    excel_sheet=pd.read_excel('Student Gradebook.xlsx',sheet_name=None,usecols=range(0,10))
    list_data=[]
    for i in excel_sheet:
        list_data.append(excel_sheet[i])
    #data_july=excel_sheet['July'].copy()
    #data_august=excel_sheet['August'].copy()
    data=pd.concat(list_data).reset_index().drop(columns=['index'])
    #del data_july
    #del data_august
    return data

In [5]:
def mydayCleaner(data):
    idx=np.array([])
    for date,month in zip(data['Date'].values,data['Month'].values):
        date_mask=data['Date']==date
        month_mask=data['Month']==month
        task_mask=data['Task']=='AjKyaUkhada'
        d=data[date_mask & month_mask & task_mask].sort_values('Student')
        idx=np.append(idx,d[d.duplicated(subset=['Student'],keep='last')].index)
    data.drop(idx,inplace=True)
    return data

In [6]:
def knowledgeSharingCleaner(data):
    idx=np.array([])
    knowledge_sharing_days={'Monday':['Kunal','Bhavna','Apurwa'],\
                       'Tuesday':['Chandrima','Purbita','Prasoon'],\
                        'Wednesday':['Roumyak','Kaushal','Ujjainee'],\
                       'Thursday':['Siddhishikha','Dipam','Shakib'],\
                        'Friday':['Sonali','Durga','Sonali'],\
                       'Saturday':['Swaastick','Sharika','Vishal'],\
                       'Sunday':['Anjali','Arya','Surabhi']} 
    date_month_dayname_unique=data[['Date','Month','DayName']].drop_duplicates()
    for date,month,dayname in zip(date_month_dayname_unique['Date'].values.tolist(),date_month_dayname_unique['Month'].values.tolist(),date_month_dayname_unique['DayName'].values.tolist()):
        dateMask=data['Date']==date
        monthMask=data['Month']==month
        daynameMask=data['DayName']==dayname
        taskMask=data['Task']=='Knowledge Sharing'
        temp=data[dateMask & monthMask & daynameMask & taskMask]

        # Below for loop filters out those rows for which students and daynames dont match
        # Example:- if dipam posts on gyan on monday, then he wont be given marks
        for index,name in zip(temp.Student.index,temp.Student.values):
            if name not in knowledge_sharing_days[dayname]:
                idx=np.append(idx,index)
        # After the first loop is run (description given on top), now it's time to filter out
        # those who put more than one post in gyan channel on the same day
        idx=np.append(idx,temp[temp.duplicated(subset=['Student'],keep='last')].index)
    data.drop(idx,inplace=True)
    return data

In [7]:
def data_cleaner(data):
    """This method will clean the data thoroughly and return the cleaned dataframe"""
    data=data.dropna(how='all',axis=0)
    data.replace('Ajkyaukhada','AjKyaUkhada',inplace=True)
    data['Date']=pd.to_datetime(data['Date'],errors='ignore',infer_datetime_format=True,dayfirst=True)
    data.dropna(subset=['Student'],inplace=True)
    data.replace('Swaastik','Swaastick',inplace=True)
    data.replace('Sushree','Siddhishikha',inplace=True)
    niteshIndex=data[data['Student']=='Nitish'].index
    data.drop(niteshIndex,inplace=True)
    data['Month']=data['Date'].dt.month_name()
    data['Year']=data['Date'].dt.year
    data['DayName']=data['Date'].dt.weekday_name
    data['Date']=data['Date'].dt.day
    data=mydayCleaner(data)
    data=knowledgeSharingCleaner(data)
    del niteshIndex
    return data

In [8]:
fileDownloader()
data=data_importer()

Download. 100


In [9]:
data.shape

(1363, 10)

In [10]:
data=data_cleaner(data)

In [11]:
data.shape

(1198, 13)

In [12]:
data

,id,Task,Date,Module,Type,Student,Late Submission,Points,Total,Task Winner,Month,Year,DayName
0,1,Linkedin Profile create/update,22,Profile Building,"Intent, Creativity,Content",Chandrima,0,6.0,10,0,July,2019,Monday
1,1,Linkedin Profile create/update,22,Profile Building,"Intent, Creativity,Content",Dipam,0,7.0,10,0,July,2019,Monday
2,1,Linkedin Profile create/update,22,Profile Building,"Intent, Creativity,Content",Durga,0,6.0,10,0,July,2019,Monday
3,1,Linkedin Profile create/update,22,Profile Building,"Intent, Creativity,Content",Kaushal,0,7.0,10,0,July,2019,Monday
4,1,Linkedin Profile create/update,22,Profile Building,"Intent, Creativity,Content",Arya,0,6.0,10,0,July,2019,Monday
5,1,Linkedin Profile create/update,22,Profile Building,"Intent, Creativity,Content",Kunal,0,8.0,10,0,July,2019,Monday
6,1,Linkedin Profile create/update,22,Profile Building,"Intent, Creativity,Content",Prasoon,0,5.0,10,0,July,2019,Monday
7,1,Linkedin Profile create/update,22,Profile Building,"Intent, Creativity,Content",Purbita,0,6.0,10,0,July,2019,Monday
8,1,Linkedin Profile create/update,22,Profile Building,"Intent, Creativity,Content",Roumyak,0,6.0,10,0,July,2019,Monday
9,1,Linkedin Profile create/update,22,Profile Building,"Intent, Creativity,Content",Shakib,0,6.0,10,0,July,2019,Monday


In [13]:
def get_myday(name,data,month_name=None):
    if name==None:
        raise ValueError('No name provided')
    if month_name==None:
        myday='AjKyaUkhada'
        myday_marks=data[data['Task']==myday].groupby('Student')['Points'].sum().sort_values(ascending=False).reset_index()
        temp=myday_marks[myday_marks['Student']==name]['Points'].values
        if len(temp)>0:
            return temp[0]
        else:
            return 0
    elif month_name in months:
        myday='AjKyaUkhada'
        myday_marks=data[(data['Task']==myday) & (data['Month']==month_name)].groupby('Student')['Points'].sum().sort_values(ascending=False).reset_index()
        temp=myday_marks[myday_marks['Student']==name]['Points'].values
        if len(temp)>0:
            return temp[0]
        else:
            return 0
    else:
        raise ValueError('month_name has wrong spelling supplied')

In [14]:
def get_task(name,data,month_name=None):
    knowledge='Knowledge Sharing'
    myday='AjKyaUkhada'
    if month_name==None:
        task_marks=data[(data['Task']!=knowledge) & (data['Task']!=myday)].groupby('Student')['Points'].\
        sum().sort_values(ascending=False).reset_index()
        temp=task_marks[task_marks['Student']==name]['Points'].values
        if len(temp)>0:
            return temp[0]
        else:
            return 0
    elif month_name in months:
        task_marks=data[(data['Task']!=knowledge) & (data['Task']!=myday) & (data['Month']==month_name)].groupby('Student')['Points'].\
        sum().sort_values(ascending=False).reset_index()
        temp=task_marks[task_marks['Student']==name]['Points'].values
        if len(temp)>0:
            return temp[0]
        else:
            return 0
    else:
        raise ValueError('month_name has wrong spelling supplied')

In [15]:
def get_gyan(name,data,month_name=None):
    if month_name==None:
        knowledge_marks=data[data['Task']=='Knowledge Sharing'].groupby('Student')['Points'].sum().sort_values(ascending=False).\
        reset_index()
        temp=knowledge_marks[knowledge_marks['Student']==name]['Points'].values
        if len(temp)>0:
            return temp[0]
        else:
            return 0
    elif month_name in months:
        knowledge_marks=data[(data['Task']=='Knowledge Sharing') & (data['Month']==month_name)].groupby('Student')['Points'].sum().sort_values(ascending=False).\
        reset_index()
        temp=knowledge_marks[knowledge_marks['Student']==name]['Points'].values
        if len(temp)>0:
            return temp[0]
        else:
            return 0
    else:
        raise ValueError('month_name has wrong spelling supplied')

In [16]:
def get_count_late_submission(name,data,month_name=None):
    if month_name==None:
        count=data[data['Late Submission']==1].groupby('Student')['Late Submission'].count().sort_values(ascending=False).\
        reset_index()
        temp=count[count['Student']==name]['Late Submission'].values
        if len(temp)>0:
            return temp[0]
        else:
            return 0
    elif month_name in months:
        count=data[(data['Late Submission']==1) & (data['Month']==month_name)].groupby('Student')['Late Submission'].count().\
        reset_index()
        temp=count[count['Student']==name]['Late Submission'].values
        if len(temp)>0:
            return temp[0]
        else:
            return 0
    else:
        raise ValueError('Month wrongly spelled')

In [17]:
data

,id,Task,Date,Module,Type,Student,Late Submission,Points,Total,Task Winner,Month,Year,DayName
0,1,Linkedin Profile create/update,22,Profile Building,"Intent, Creativity,Content",Chandrima,0,6.0,10,0,July,2019,Monday
1,1,Linkedin Profile create/update,22,Profile Building,"Intent, Creativity,Content",Dipam,0,7.0,10,0,July,2019,Monday
2,1,Linkedin Profile create/update,22,Profile Building,"Intent, Creativity,Content",Durga,0,6.0,10,0,July,2019,Monday
3,1,Linkedin Profile create/update,22,Profile Building,"Intent, Creativity,Content",Kaushal,0,7.0,10,0,July,2019,Monday
4,1,Linkedin Profile create/update,22,Profile Building,"Intent, Creativity,Content",Arya,0,6.0,10,0,July,2019,Monday
5,1,Linkedin Profile create/update,22,Profile Building,"Intent, Creativity,Content",Kunal,0,8.0,10,0,July,2019,Monday
6,1,Linkedin Profile create/update,22,Profile Building,"Intent, Creativity,Content",Prasoon,0,5.0,10,0,July,2019,Monday
7,1,Linkedin Profile create/update,22,Profile Building,"Intent, Creativity,Content",Purbita,0,6.0,10,0,July,2019,Monday
8,1,Linkedin Profile create/update,22,Profile Building,"Intent, Creativity,Content",Roumyak,0,6.0,10,0,July,2019,Monday
9,1,Linkedin Profile create/update,22,Profile Building,"Intent, Creativity,Content",Shakib,0,6.0,10,0,July,2019,Monday


In [18]:
def get_highest_marks(name,data,month_name=None):
    if name==None:
        raise ValueError('Name:',name)
    if month_name==None:
        temp=data.groupby('Student')['Total'].sum().reset_index()
        temp=temp[temp['Student']==name]['Total'].values
        if len(temp)>0:
            return temp[0]
        else:
            return 0
    elif month_name in months:
        temp=data[data['Month']==month_name].groupby('Student')['Total'].sum().reset_index()
        temp=temp[temp['Student']==name]['Total'].values
        if len(temp)>0:
            return temp[0]
        else:
            return 0
    else:
        raise ValueError('month spelled wrong')

In [19]:
def get_count_task_won(name,data,month_name=None):
    if month_name==None:
        count=data[data['Task Winner']==1].groupby('Student')['Task Winner'].count().sort_values(ascending=False).\
        reset_index()
        temp=count[count['Student']==name]['Task Winner'].values
        if len(temp)>0:
            return temp[0]
        else:
            return 0
    elif month_name in months:
        count=data[(data['Task Winner']==1) & (data['Month']==month_name)].groupby('Student')['Task Winner'].count().\
        reset_index()
        temp=count[count['Student']==name]['Task Winner'].values
        if len(temp)>0:
            return temp[0]
        else:
            return 0
    else:
        raise ValueError('Month wrongly spelled')

In [20]:
# Total marks function (whole)
def total_marks_leaderboard(data,month_name=None):
    """Returns a leaderboard of students containing data such as :- total, task, myday, knowledge sharing marks and
    count of late submissions and task won.
    Arguements:-
    :data - dataframe to work with
    :month_name - [Default: None] if any other value then returns leaderboard for the specified month. Make sure first
    letter of month is capital.
    
    Returns:
    tuple:- (dataframe object,dataframe with hidden indexes)"""
    if month_name==None:
        total_marks=data.groupby('Student')['Points'].sum().sort_values(ascending=False).reset_index()
        total_marks.rename(columns={'Points':'Marks'})
        total_marks['Total Marks']=total_marks['Student'].apply(get_highest_marks,args=[data])
        total_marks['Task Marks']=total_marks['Student'].apply(get_task,args=[data])
        total_marks['MyDay Marks']=total_marks['Student'].apply(get_myday,args=[data])
        total_marks['Knowledge Sharing Marks']=total_marks['Student'].apply(get_gyan,args=[data])
        total_marks['Number of Late Submissions']=total_marks['Student'].apply(get_count_late_submission,args=[data])
        total_marks['Number of Tasks Won']=total_marks['Student'].apply(get_count_task_won,args=[data])
        return tuple((total_marks,total_marks.style.hide_index()))
    elif month_name in months:
        temp=data[data['Month']==month_name]
        total_marks=data[data['Month']==month_name].groupby('Student')['Points'].sum().sort_values(ascending=False).reset_index()
        total_marks.rename(columns={'Points':'Marks'})
        total_marks['Total Marks']=total_marks['Student'].apply(get_highest_marks,args=[temp,month_name])
        total_marks['Task Marks']=total_marks['Student'].apply(get_task,args=[temp,month_name])
        total_marks['MyDay Marks']=total_marks['Student'].apply(get_myday,args=[temp,month_name])
        total_marks['Knowledge Sharing Marks']=total_marks['Student'].apply(get_gyan,args=[temp,month_name])
        total_marks['Number of Late Submissions']=total_marks['Student'].apply(get_count_late_submission,args=[temp,month_name])
        total_marks['Number of Tasks Won']=total_marks['Student'].apply(get_count_task_won,args=[temp,month_name])
        return tuple((total_marks,total_marks.style.hide_index()))
    else:
        raise ValueError('month_name has wrong spelling supplied')

In [21]:
total_marks_leaderboard(data)[1]

Student,Points,Total Marks,Task Marks,MyDay Marks,Knowledge Sharing Marks,Number of Late Submissions,Number of Tasks Won
Ujjainee,301,328,188,103,10,3,0
Shakib,293,323,185,99,9,4,3
Surabhi,291,321,185,103,3,4,1
Kunal,286.5,306,175.5,105,6,4,3
Vishal,281.5,313,183.5,96,2,2,2
Chandrima,279.5,327,167.5,101,11,6,3
Durga,279.5,328,176.5,98,5,5,1
Sonali,275.5,313,177.5,89,9,5,2
Dipam,273.5,321,167.5,103,3,2,1
Purbita,270.5,299,166.5,95,9,5,2


In [22]:
total_marks_leaderboard(data,month_name='July')[0]

,Student,Points,Total Marks,Task Marks,MyDay Marks,Knowledge Sharing Marks,Number of Late Submissions,Number of Tasks Won
0,Kunal,78.5,91,62.5,16.0,0,2,1
1,Ujjainee,75.5,91,59.5,16.0,0,1,0
2,Purbita,75.0,91,59.0,16.0,0,1,1
3,Surabhi,72.5,89,58.5,14.0,0,1,0
4,Vishal,72.0,91,56.0,16.0,0,0,0
5,Sharika,70.0,91,54.0,16.0,0,1,0
6,Prasoon,69.5,87,57.5,12.0,0,1,0
7,Roumyak,69.0,89,55.0,14.0,0,0,0
8,Shakib,69.0,89,55.0,14.0,0,0,0
9,Kaushal,68.5,89,54.5,14.0,0,1,1


In [23]:
def student_task_marks(data,name=None,month_name=None):
    """Returns the task and points of the given student name in DataFrame
    Arguments:
        data: dataframe from which marks need to be extracted
        name: name of the student, only the first name and first letter capital.
        monthWise: default: False
                    set to True if dataframe needed only for a particular month
        month_name: default None.
                    if monthWise True and month_name is name of the month then
                    Dataframe of that particular month is given.
                    
    Returns:
        tuple:- (dataframe object,dataframe with hidden indexes)
    """
    if name==None:
        raise ValueError('Input a Name')
    if month_name!=None:
        if month_name in months:
            task_marks=data[(data['Task']!='Knowledge Sharing') & (data['Task']!='AjKyaUkhada') & (data['Student']==name) & \
                       (data['Month']==month_name)]
        else:
            raise ValueError('Wrong month spelling given')
    elif month_name==None:
        task_marks=data[(data['Task']!='Knowledge Sharing') & (data['Task']!='AjKyaUkhada') & (data['Student']==name)]
    task_marks=task_marks[['Date','Month','Task','Student','Points','Total','Late Submission','Task Winner']]
    task_marks['Late Submission'].replace([0,1],['No','Yes'],inplace=True)
    task_marks['Task Winner'].replace([0,1],['No','Yes'],inplace=True)
    return tuple((task_marks,task_marks.style.hide_index()))

In [29]:
student_task_marks(data,name='Kunal')[1]

Date,Month,Task,Student,Points,Total,Late Submission,Task Winner
22,July,Linkedin Profile create/update,Kunal,8,10,No,No
23,July,Blog on Machine Learning and it's impact,Kunal,21,25,No,No
30,July,Blog on Different data based job roles,Kunal,21.5,25,Yes,Yes
31,July,Portfolio home page,Kunal,12,15,Yes,No
1,August,Video Resume,Kunal,20,20,No,No
8,March,Online Resume,Kunal,18,20,No,No
8,April,Blog and Project Pages,Kunal,9.5,10,No,No
13,August,Github Profile Creation,Kunal,9.5,10,No,Yes
14,August,PPT Submission,Kunal,5,5,Yes,No
23,August,Blog on Data Analysis Process,Kunal,26,30,No,No


In [25]:
# Student myday count and marks
def student_myday(data,name=None,month_name=None):
    """Gets the myday marks of the given student
    Arguements:
    name - name of the student
    data - dataframe to work with
    month_name - [Default: None] name of month to check the dataframe for.
    
    Returns:
    tuple:- (dataframe object,dataframe with hidden indexes)"""
    myday='AjKyaUkhada'
    if name==None:
        raise ValueError('Input a name')
    if month_name==None:
        myday_marks=data[(data['Task']==myday) & (data['Student']==name)].groupby('Student')['Points'].sum().sort_values(ascending=False).reset_index()
        myday_marks.rename(columns={'Points':'Myday Marks'},inplace=True)
        myday_count=data[(data['Task']==myday) & (data['Student']==name)].groupby('Student')['Points'].count().sort_values(ascending=False).reset_index()
        myday_count.rename(columns={'Points':'Number of days'},inplace=True)
        myday_data=myday_marks.merge(myday_count,on='Student')
        return tuple((myday_data,myday_data.style.hide_index()))
    elif month_name in months:
        myday_marks=data[(data['Month']==month_name) & (data['Task']==myday) & (data['Student']==name)].groupby('Student')['Points'].sum().sort_values(ascending=False).reset_index()
        myday_marks.rename(columns={'Points':'Myday Marks'},inplace=True)
        myday_count=data[(data['Month']==month_name) & (data['Task']==myday) & (data['Student']==name)].groupby('Student')['Points'].count().sort_values(ascending=False).reset_index()
        myday_count.rename(columns={'Points':'Number of days'},inplace=True)
        myday_data=myday_marks.merge(myday_count,on='Student')
        return tuple((myday_data,myday_data.style.hide_index()))

In [26]:
student_myday(data,name='Kunal',month_name='September')[0]

,Student,Myday Marks,Number of days
0,Kunal,27.0,18


In [27]:
def student_knowledge_marks(data,name=None,month_name=None):
    """Gets the knowledge sharing marks of the given student
    Arguements:
    name - name of the student
    data - dataframe to work with
    month_name - [Default: None] name of month to check the dataframe for.
    
    Returns:
    tuple:- (dataframe object,dataframe with hidden indexes)"""
    knowledge='Knowledge Sharing'
    if name==None:
        raise ValueError('Input a name')
    if month_name==None:
        knowledge_marks=data[(data['Task']==knowledge) & (data['Student']==name)].groupby('Student')['Points'].sum().sort_values(ascending=False).reset_index()
        knowledge_marks.rename(columns={'Points':'Knowledge Sharing Marks'},inplace=True)
        knowledge_count=data[(data['Task']==knowledge) & (data['Student']==name)].groupby('Student')['Points'].count().sort_values(ascending=False).reset_index()
        knowledge_count.rename(columns={'Points':'Number of days'},inplace=True)
        knowledge_data=knowledge_marks.merge(knowledge_count,on='Student')
        return tuple((knowledge_data,knowledge_data.style.hide_index()))
    elif month_name in months:
        knowledge_marks=data[(data['Month']==month_name) & (data['Task']==knowledge) & (data['Student']==name)].groupby('Student')['Points'].sum().sort_values(ascending=False).reset_index()
        knowledge_marks.rename(columns={'Points':'Knowledge Sharing Marks'},inplace=True)
        knowledge_count=data[(data['Month']==month_name) & (data['Task']==knowledge) & (data['Student']==name)].groupby('Student')['Points'].count().sort_values(ascending=False).reset_index()
        knowledge_count.rename(columns={'Points':'Number of days'},inplace=True)
        knowledge_data=knowledge_marks.merge(knowledge_count,on='Student')
        return tuple((knowledge_data,knowledge_data.style.hide_index()))

In [28]:
student_knowledge_marks(data,'Kunal','September')[0]

,Student,Knowledge Sharing Marks,Number of days
0,Kunal,2.0,1
